In [ ]:
# Setup and Dependencies
# Input Protein Sequence
# Protein Structural Modeling with ESMFold (ESM-3)
# B-cell Epitope Prediction
# Linear Epitope Prediction
# Conformational Epitope Prediction
# T-cell Epitope Prediction
# Population Coverage Analysis
# Molecular Docking Simulation
# Visualization and Analysis

In [2]:
# import os
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
from Bio import SeqIO
# from Bio.SeqUtils.ProtParam import ProteinAnalysis
# from Bio.PDB import PDBParser, Selection
# import torch
# import esm  # For ESM-3 and ESMFold
# Additional imports for visualization and docking

In [3]:
def load_fasta(file_path):
    """
    Load protein sequences from a FASTA file.
    """
    sequences = []
    for record in SeqIO.parse(file_path, "fasta"):
        sequences.append(str(record.seq))
    return sequences


# Example usage:
sequences = load_fasta("../allergen-analysis/ara-h-2/AAN77576.fasta")
protein_sequence = sequences[0]  # Use the first sequence for this example

In [4]:
import os

from dotenv import load_dotenv
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig
from huggingface_hub import login

# Load the environment variables from the .env file
load_dotenv()

# Retrieve the Hugging Face token from the environment
hf_token = os.getenv("HUGGINGFACE_TOKEN")

# Will instruct you how to get an API key from huggingface hub, make one with "Read" permission.
login(token=hf_token)

# This will download the model weights and instantiate the model on your machine.
model: ESM3InferenceClient = ESM3.from_pretrained("esm3_sm_open_v1").to(
    "cuda"
)  # or "cpu"

# Generate a completion for a partial Carbonic Anhydrase (2vvb)
prompt = "___________________________________________________DQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPP___________________________________________________________"
protein = ESMProtein(sequence=prompt)
# Generate the sequence, then the structure. This will iteratively unmask the sequence track.
protein = model.generate(
    protein, GenerationConfig(track="sequence", num_steps=8, temperature=0.7)
)
# We can show the predicted structure for the generated sequence.
protein = model.generate(protein, GenerationConfig(track="structure", num_steps=8))
protein.to_pdb("./generation.pdb")
# Then we can do a round trip design by inverse folding the sequence and recomputing the structure
protein.sequence = None
protein = model.generate(protein, GenerationConfig(track="sequence", num_steps=8))
protein.coordinates = None
protein = model.generate(protein, GenerationConfig(track="structure", num_steps=8))
protein.to_pdb("./round_tripped.pdb")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/atom/.cache/huggingface/token
Login successful


/home/atom/Projects/BioxML-allergen-epitopes/notebooks/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

data/entry_list_safety_29026.list:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

...

esm3_sm_open_v1.pth:   0%|          | 0.00/2.80G [00:00<?, ?B/s]

esm3_structure_decoder_v0.pth:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

esm3_structure_encoder_v0.pth:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

/home/atom/Projects/BioxML-allergen-epitopes/notebooks/.venv/lib/python3.12/site-packages/esm/pretrained.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = to

KeyboardInterrupt: 